Load dataset into Python

In [1]:
!git clone https://github.com/gsukuma/mlproject.git

Cloning into 'mlproject'...
remote: Enumerating objects: 35122, done.
remote: Counting objects: 100% (1067/1067), done.
remote: Compressing objects: 100% (1065/1065), done.
remote: Total 35122 (delta 1), reused 1066 (delta 0), pack-reused 34055
Receiving objects: 100% (35122/35122), 195.91 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (36985/36985), done.


Load necessary libraries

In [2]:
import numpy as np
from imutils import paths
import os
import matplotlib.pyplot as plt

In [3]:
#Load image path
imagePaths = list(paths.list_images('/content/mlproject/dataset_test/archive/images/train'))

In [4]:
imagePaths[0]

'/content/mlproject/dataset_test/archive/images/train/disgust/6145.jpg'

In [5]:
label = imagePaths[0].split(os.path.sep)[-2]
label

'disgust'

In [13]:
data = []
labels = []
IMG_SIZE = 48

CHANNELS = 3 

for imagePath in imagePaths:
  label = imagePath.split(os.path.sep)[-2]
  image = load_img(imagePath, target_size=(IMG_SIZE, IMG_SIZE))
  image = img_to_array(image)
  image = image/255

  data.append(image)
  labels.append(label)


data = np.array(data, dtype="float32")
labels = np.array(labels)

In [21]:
data.shape

(28821, 48, 48, 3)

In [22]:
labels.shape

(28821,)

In [16]:
#Load validation data
imageValPaths = list(paths.list_images('/content/mlproject/dataset_test/archive/images/validation'))

In [20]:
valdata = []
vallabels = []
IMG_SIZE = 48

CHANNELS = 3 

for imagePath in imageValPaths:
  vallabel = imagePath.split(os.path.sep)[-2]
  image = load_img(imagePath, target_size=(IMG_SIZE, IMG_SIZE))
  image = img_to_array(image)
  image = image/255

  valdata.append(image)
  vallabels.append(vallabel)


valdata = np.array(valdata, dtype="float32")
vallabels = np.array(vallabels)

In [23]:
valdata.shape

(7066, 48, 48, 3)

In [24]:
vallabels.shape

(7066,)

In [25]:
np. unique(labels, return_counts=True)

(array(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
       dtype='<U8'), array([3993,  436, 4103, 7164, 4982, 4938, 3205]))

In [26]:
np. unique(vallabels, return_counts=True)

(array(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
       dtype='<U8'), array([ 960,  111, 1018, 1825, 1216, 1139,  797]))

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)
#one hot encoding
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

In [30]:
labels[0]

array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [34]:
vallabels = le.fit_transform(vallabels)
vallabels = to_categorical(vallabels)

In [35]:
vallabels[0]

array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)

Model Definition

In [40]:
import tensorflow as tf

In [41]:
#Model 1
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam

# Model parameters
num_classes = 7
input_shape = (48,48,3)

# Define the model architecture
input_layer = Input(shape=input_shape)

x = Conv2D(32, (3,3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Flatten()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

output_layer = Dense(num_classes, activation='softmax')(x)

# Compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  loss="categorical_crossentropy",
  metrics=["accuracy"])

In [42]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=25, verbose=1, restore_best_weights=True, min_delta=0.01, 
                              baseline=0.6)

model.fit(data, labels, epochs=25, batch_size=300, validation_data=(valdata,vallabels), callbacks=[early_stopping])

Epoch 1/25
97/97 [==============================] - 8s 35ms/step - loss: 1.8837 - accuracy: 0.2177 - val_loss: 1.8310 - val_accuracy: 0.2583
Epoch 2/25
97/97 [==============================] - 3s 27ms/step - loss: 1.8506 - accuracy: 0.2324 - val_loss: 1.8190 - val_accuracy: 0.2583
Epoch 3/25
97/97 [==============================] - 3s 29ms/step - loss: 1.8431 - accuracy: 0.2353 - val_loss: 1.8144 - val_accuracy: 0.2583
Epoch 4/25
97/97 [==============================] - 3s 27ms/step - loss: 1.8367 - accuracy: 0.2364 - val_loss: 1.8090 - val_accuracy: 0.2584
Epoch 5/25
97/97 [==============================] - 3s 27ms/step - loss: 1.8324 - accuracy: 0.2372 - val_loss: 1.8041 - val_accuracy: 0.2583
Epoch 6/25
97/97 [==============================] - 3s 27ms/step - loss: 1.8294 - accuracy: 0.2409 - val_loss: 1.7999 - val_accuracy: 0.2583
Epoch 7/25
97/97 [==============================] - 3s 28ms/step - loss: 1.8230 - accuracy: 0.2438 - val_loss: 1.7952 - val_accuracy: 0.2594
Epoch 8/25
97

In [43]:
model.save('FaceRecognition_model1.h5')

In [51]:
#Transfer learning with RestNet50
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input
feature_extractor_layer = ResNet50V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(IMG_SIZE,IMG_SIZE,CHANNELS)))
#Neural networks start from bottom to top. include_top=false will remove the bottom layers


94668760/94668760 [==============================] - 5s 0us/step


In [52]:
feature_extractor_layer.trainable = False

In [64]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

model = tf.keras.Sequential()
model.add(feature_extractor_layer)
model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(1024, activation='relu', name='hidden_layer'))
model.add(layers.Dropout(0.5))#Dropping 50% to avoid overfitting
model.add(layers.Dense(7, activation='softmax', name='output'))
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2, 2, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 hidden_layer (Dense)        (None, 1024)              8389632   
                                                                 
 dropout_22 (Dropout)        (None, 1024)              0         
                                                                 
 output (Dense)              (None, 7)                 7175      
                                                                 
Total params: 31,961,607
Trainable params: 8,396,807
Non-trainable params: 23,564,800
_________________________________________________________________


In [65]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  loss="categorical_crossentropy",
  metrics=["accuracy"])

In [55]:
#Data augmentation. Add a few impurities
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [66]:
history = model.fit(data, labels,
                    validation_data=(valdata,vallabels),
                    batch_size=300,
                    epochs=50,
                    verbose=1)

Epoch 1/50
97/97 [==============================] - 14s 116ms/step - loss: 3.7092 - accuracy: 0.2059 - val_loss: 2.0326 - val_accuracy: 0.3036
Epoch 2/50
97/97 [==============================] - 9s 90ms/step - loss: 2.9104 - accuracy: 0.2493 - val_loss: 1.8681 - val_accuracy: 0.3309
Epoch 3/50
97/97 [==============================] - 9s 88ms/step - loss: 2.4834 - accuracy: 0.2737 - val_loss: 1.7696 - val_accuracy: 0.3442
Epoch 4/50
97/97 [==============================] - 8s 88ms/step - loss: 2.2000 - accuracy: 0.2967 - val_loss: 1.7112 - val_accuracy: 0.3592
Epoch 5/50
97/97 [==============================] - 7s 75ms/step - loss: 2.0133 - accuracy: 0.3132 - val_loss: 1.6724 - val_accuracy: 0.3704
Epoch 6/50
97/97 [==============================] - 7s 74ms/step - loss: 1.8910 - accuracy: 0.3331 - val_loss: 1.6430 - val_accuracy: 0.3766
Epoch 7/50
97/97 [==============================] - 7s 75ms/step - loss: 1.7894 - accuracy: 0.3473 - val_loss: 1.6229 - val_accuracy: 0.3849
Epoch 8/50


In [67]:
model.save('FaceRecognition_model2.h5', save_format="h5")